## Autograph使用规范

1、被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数

In [14]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [15]:
# np_random 每次执行都是一样的结果
np_random()
np_random()

array([[ 1.54702915, -1.77596667,  1.39696143],
       [-0.70044697,  1.20457398,  0.16992572],
       [-0.59155276,  1.5922341 ,  0.36344772]])
array([[ 1.54702915, -1.77596667,  1.39696143],
       [-0.70044697,  1.20457398,  0.16992572],
       [-0.59155276,  1.5922341 ,  0.36344772]])


In [16]:
np.random.randn(3,3)

array([[ 0.41895959,  0.59207704,  1.32041083],
       [-0.41126656, -0.93646575, -0.18559257],
       [ 0.34970425, -0.13584612, -0.36640597]])

In [17]:
np.random.randn(3,3)

array([[ 0.50624242,  0.55384901,  0.5254847 ],
       [ 1.0053737 , -1.64728958, -1.47254572],
       [ 1.69605283, -0.24640298,  0.33859674]])

In [18]:
tf_random()
tf_random()

[[1.31276178 0.103609778 0.541209579]
 [-1.04489613 -0.59034 -0.384477675]
 [-0.821616769 0.0505642034 -0.779052138]]
[[0.702456534 0.187934354 0.975813925]
 [-0.518767715 -0.32689631 -0.281255782]
 [0.892493606 0.298080713 -0.0819215]]


2、避免在@tf.function修饰的函数内部定义tf.Variable

In [19]:
x = tf.Variable(1.0, dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var()
outer_var()

2
3


<tf.Tensor: id=149, shape=(), dtype=float32, numpy=3.0>

In [20]:
# 报错
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return (x)

# inner_var() #打开重现报错

3、被@tf.function修饰的函数不可修改，该函数外部的Python列表或字典等结构类型变量

In [21]:
tensor_list = []
# @tf.function # 加上这一行切换成Autograph结果不符合预期
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: id=150, shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: id=151, shape=(), dtype=float32, numpy=6.0>]


In [22]:
tensor_list = []

@tf.function #加上这一行切换成Autograph结果不符合预期
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
